# ELISA - Explore the Data

## Python Environment

In [1]:
import os
import pandas as pd
import seaborn as sns
import mysql.connector
from sqlalchemy import create_engine
from sqlalchemy import (Column, DateTime, Integer, Numeric, Float, String, Text)

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

from pathlib import Path
from dotenv import load_dotenv

from IPython.display import HTML, display

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

from IPython.core.interactiveshell import InteractiveShell

## User Environment

In [2]:
env_path = Path('.')/'.env'
load_dotenv(".env")
DR_USER = os.environ.get('DR_USER')
DR_PASSWORD = os.environ.get('DR_PASSWORD')
DR_SERVER = os.environ.get('DR_SERVER')
DR_DATABASE = os.environ.get('DR_DATABASE')
SHARED_USER = os.environ.get('SHARED_USER')
SHARED_PASSWORD = os.environ.get('SHARED_PASSWORD')
SHARED_SERVER = os.environ.get('SHARED_SERVER')
SHARED_DATABASE = os.environ.get('SHARED_DATABASE')
DATA_DIRECTORY = os.environ.get('DATA_DIRECTORY')

# Use the flag below to turn off or turn on the running of the SQL that
# interacts with MySQL. This is being used to hopefully allow the 
# execution of the notebook code, when you do not have a connection to
# MySQL.
RUN_SQL = True 
GEN_PLOTS = False

## Database Information

To explore and clean up the result information, we will extract data from the latest ImmPort DataRelease(DR) database, and then store the temporary and final results in a SharedData database. Much of this work and exploration could be done entirely in the notebook without the SharedData database, but we may use the SharedData database as a source for dynamic web content.

First step is to creat the SharedData database if it does not already exist. To create the new database login to the MySQL server then run the command below:

<div class="alert alert-block alert-info">
create database SharedData character set utf8
</div>

Next we will setup connections to the DR database and the SharedData database

In [3]:
conn1 = f"mysql+mysqlconnector://{DR_USER}:{DR_PASSWORD}@{DR_SERVER}:3306/{DR_DATABASE}"
dr_conn = create_engine(conn1, echo=False)
conn2 = f"mysql+mysqlconnector://{SHARED_USER}:{SHARED_PASSWORD}@{SHARED_SERVER}:3306/{SHARED_DATABASE}"
shared_conn = create_engine(conn2, echo=False)

# Initialize the ShareData ELISA_Results Table

Clone the elisa_result table, with additional columns to the Shared_Database. We will use this version of the table to explore the contents of this table, plus using SQL statements we will clean up the data for data analysis.

In [4]:
if RUN_SQL:
    sql = f'''
CREATE TABLE {SHARED_DATABASE}.elisa_result AS
(
SELECT l1.immunology_symbol,
       e1.analyte_reported,
       e1.unit_reported,
       e1.unit_preferred,
       e1.value_reported,
       e1.value_preferred,
       e1.study_accession,
       e1.arm_accession,
       e1.subject_accession,
       e1.biosample_accession,
       e1.experiment_accession,
       e1.expsample_accession,
       e1.study_time_collected,
       e1.study_time_collected_unit,
       b1.type as biosample_type,
       b1.planned_visit_accession,
       p1.name as planned_visit_name,
       p1.min_start_day,
       p1.max_start_day
  FROM elisa_result e1
    JOIN biosample b1
      ON e1.biosample_accession = b1.biosample_accession
    JOIN planned_visit p1
      ON b1.planned_visit_accession = p1.planned_visit_accession
    LEFT OUTER JOIN lk_analyte l1
      ON e1.analyte_preferred = l1.analyte_accession
)
'''
    shared_conn.execute('DROP TABLE IF EXISTS elisa_result')
    dr_conn.execute(sql)
    shared_conn.execute('CREATE INDEX idx_elisa_study_accession on elisa_result(study_accession)')
    shared_conn.execute('CREATE INDEX idx_elisa_arm_accession on elisa_result(arm_accession)')
    shared_conn.execute('CREATE INDEX idx_elisa_biosample_accession on elisa_result(biosample_accession)')
    shared_conn.execute('CREATE INDEX idx_elisa_experiment_accession on elisa_result(experiment_accession)')
    shared_conn.execute('CREATE INDEX idx_elisa_expsample_accession on elisa_result(expsample_accession)')
    shared_conn.execute('CREATE INDEX idx_elisa_subject_accession on elisa_result(subject_accession);')
    

# Explore Current ELISA Results Content

Review summary information on the content in specific columns in the Elisa_result table. The kinds of information we are looking for are the number of rows in the table; have the unit_preferred and value_preferred columns been populated, what are the different unit_preferred values, etc.

## Row Counts

In [5]:
if RUN_SQL:
    sql = '''
SELECT count(*)
  FROM elisa_result
'''
    count = shared_conn.execute(sql).fetchone()[0]
    print("Total Rows: ",count)

    sql = '''
SELECT count(*)
  FROM elisa_result
 WHERE unit_reported is not null'''
    count = shared_conn.execute(sql).fetchone()[0]
    print("Total Rows - unit_reported: ",count)

    sql = '''
SELECT count(*)
  FROM elisa_result
 WHERE unit_preferred is not null'''
    count = shared_conn.execute(sql).fetchone()[0]
    print("Total Rows - unit_preferred: ",count)

    sql = '''
SELECT count(*)
  FROM elisa_result
 WHERE value_reported is not null'''
    count = shared_conn.execute(sql).fetchone()[0]
    print("Total Rows - value_reported: ",count)

    sql = '''
SELECT count(*)
  FROM elisa_result
 WHERE value_preferred is not null'''
    count = shared_conn.execute(sql).fetchone()[0]
    print("Total Rows - value_preferred: ",count)

    sql = '''
SELECT count(*)
  FROM elisa_result
 WHERE analyte_reported is not null'''
    count = shared_conn.execute(sql).fetchone()[0]
    print("Total Rows - analyte_reported: ",count)

    sql = '''
SELECT count(*)
  FROM elisa_result
 WHERE immunology_symbol is not null'''
    count = shared_conn.execute(sql).fetchone()[0]
    print("Total Rows - analyte_preferred: ",count)

Total Rows:  246120
Total Rows - unit_reported:  246120
Total Rows - unit_preferred:  92555
Total Rows - value_reported:  245862
Total Rows - value_preferred:  245862
Total Rows - analyte_reported:  246120
Total Rows - analyte_preferred:  200679


# QC and Clean Data

## Check the Unit Information
Based on work for the 10K project, we will convert all the rows we can to the standard unit of pg/ml. We will then remove all the rows that are not in this standard unit. This section will need to be reviewed, to see if other conversions are possible, since many rows will be removed.

In [6]:
InteractiveShell.ast_node_interactivity = "last"

In [7]:
if RUN_SQL:
    sql = '''
SELECT BINARY e1.unit_reported, e1.unit_reported, count(*)
  FROM elisa_result e1
  GROUP BY BINARY e1.unit_reported, e1.unit_reported
  ORDER BY BINARY e1.unit_reported, e1.unit_reported
'''
    unit_reported = pd.read_sql(sql, shared_conn)
    number_of_rows = len(unit_reported)
    pd.set_option("display.max_rows",number_of_rows)
    print(unit_reported.shape)
    unit_reported.head(number_of_rows)

(48, 3)


,BINARY e1.unit_reported,unit_reported,count(*)
0,b'-1: neg 1: pos',-1: neg 1: pos,49
1,b'1.00E+01',1.00E+01,17
2,b'1.00E+02',1.00E+02,17
3,b'1.00E+03',1.00E+03,16
4,b'1.00E+04',1.00E+04,17
5,b'1.00E+05',1.00E+05,17
6,b'1.00E+06',1.00E+06,17
7,b'1/OD 450nm',1/OD 450nm,115
8,b'405 nm',405 nm,304
9,b'AU/ml',AU/ml,282


In [8]:
if RUN_SQL:
    sql = '''
SELECT BINARY e1.unit_preferred, e1.unit_preferred, count(*)
  FROM elisa_result e1
  GROUP BY BINARY e1.unit_preferred, e1.unit_preferred
  ORDER BY BINARY e1.unit_preferred, e1.unit_preferred
'''
    unit_preferred = pd.read_sql(sql, shared_conn)
    number_of_rows = len(unit_preferred)
    pd.set_option("display.max_rows",number_of_rows)
    print(unit_preferred.shape)
    unit_preferred.head(number_of_rows)

(4, 3)


,BINARY e1.unit_preferred,unit_preferred,count(*)
0,None,None,153565
1,"[110, 103, 47, 109, 108]",ng/ml,15104
2,"[112, 103, 47, 109, 108]",pg/ml,56393
3,"[117, 103, 47, 109, 108]",ug/ml,21058


## Convert the value_preferred and unit_preferred to Standard Units

For the 10K data, Kelly converted the unit_preferred to pg/ml and then ran conversion scripts to convert the value_preferred into the pg/ml unit.

In [9]:
InteractiveShell.ast_node_interactivity = "none"

In [10]:
if RUN_SQL:
    sql = '''
UPDATE elisa_result SET unit_preferred = 'pg/ml'
    WHERE unit_reported = 'pg/ml'
      AND unit_preferred is null;
'''
    shared_conn.execute(sql);

    sql = '''
UPDATE elisa_result SET value_preferred = (1000 * value_preferred)
    WHERE unit_preferred = 'ng/ml';
'''
    shared_conn.execute(sql);

    sql = '''
UPDATE elisa_result SET unit_preferred = 'pg/ml'
    WHERE unit_preferred = 'ng/ml';
'''
    shared_conn.execute(sql);

    sql = '''
UPDATE elisa_result SET value_preferred = (10000 * value_preferred)
    WHERE unit_preferred = 'ug/dl';
'''
    shared_conn.execute(sql);

    sql = '''
UPDATE elisa_result SET unit_preferred = 'pg/ml'
    WHERE unit_preferred = 'ug/dl';
'''
    shared_conn.execute(sql);

    sql = '''
UPDATE elisa_result SET value_preferred = (1000000 * value_preferred)
    WHERE unit_preferred = 'ug/ml';
'''
    shared_conn.execute(sql);

    sql = '''
UPDATE elisa_result SET unit_preferred = 'pg/ml'
    WHERE unit_preferred = 'ug/ml';
'''
    shared_conn.execute(sql);

    sql = '''
DELETE FROM elisa_result WHERE unit_preferred is null
'''
    shared_conn.execute(sql);

## Review Null Columns

In [11]:
if RUN_SQL:
    sql = '''
SELECT count(*)
  FROM elisa_result
'''
    count = shared_conn.execute(sql).fetchone()[0]
    print("Total Rows: ",count)
    
    sql = '''
SELECT count(*)
  FROM elisa_result
 WHERE value_preferred is not null
'''
    count = shared_conn.execute(sql).fetchone()[0]
    print("Total Rows - Value Preferred: ",count)
    
    sql = '''
SELECT count(*)
  FROM elisa_result
 WHERE immunology_symbol is not null
'''
    count = shared_conn.execute(sql).fetchone()[0]
    print("Total Rows - Analyte Preferred: ",count)

Total Rows:  166032
Total Rows - Value Preferred:  166032
Total Rows - Analyte Preferred:  133815


## Drop Rows with Null values and immunology_symbol's

In [12]:
if RUN_SQL:
   sql = '''
DELETE FROM elisa_result WHERE value_preferred is null
'''
   shared_conn.execute(sql); 
    
   sql = '''
DELETE FROM elisa_result WHERE immunology_symbol is null
'''
   shared_conn.execute(sql);

## Final Results After QC

In [13]:
if RUN_SQL:
    sql = '''
SELECT count(*)
  FROM elisa_result
'''
    count = shared_conn.execute(sql).fetchone()[0]
    print("Total Rows: ",count)
    
    sql = '''
SELECT count(*)
  FROM elisa_result
 WHERE value_preferred is not null
'''
    count = shared_conn.execute(sql).fetchone()[0]
    print("Total Rows - Value Preferred: ",count)
    
    sql = '''
SELECT count(*)
  FROM elisa_result
 WHERE immunology_symbol is not null
'''
    count = shared_conn.execute(sql).fetchone()[0]
    print("Total Rows - Analyte Preferred: ",count)

Total Rows:  133815
Total Rows - Value Preferred:  133815
Total Rows - Analyte Preferred:  133815


# Export the Clean ELISA_RESULT table and Create DataFrame

## Export ELISA_RESULT

## Create DataFrame from ELISA_RESULT Text File

In [14]:
file_name = DATA_DIRECTORY + "/elisa_result_clean.txt"
if RUN_SQL:
    sql = '''
SELECT e1.immunology_symbol,
       e1.analyte_reported,
       e1.unit_reported,
       e1.unit_preferred,
       e1.value_reported,
       e1.value_preferred,
       e1.study_accession,
       e1.arm_accession,
       e1.subject_accession,
       e1.biosample_accession,
       e1.experiment_accession,
       e1.expsample_accession,
       e1.study_time_collected,
       e1.study_time_collected_unit,
       e1.biosample_type,
       e1.planned_visit_accession,
       e1.planned_visit_name,
       e1.min_start_day,
       e1.max_start_day
  FROM elisa_result e1
'''
    
    elisa_result = pd.read_sql(sql, shared_conn)
    elisa_result.to_csv(file_name, sep='\t')

In [15]:
 elisa_result = pd.read_csv(file_name, sep="\t")


# Review Summary Information

## Review the Analyte_Preferred and Analyte_Reported Content

Below are a couple of queries, to identify the current state of the analyte and unit columns. This information may be of use in the curation process. 

In [16]:
InteractiveShell.ast_node_interactivity = "last_expr"
pd.set_option("display.max_rows",1000)

In [17]:
analyte_preferred_count = elisa_result.groupby('immunology_symbol')['immunology_symbol'].count().to_frame()
analyte_preferred_count.rename(columns={'immunology_symbol':'count'}, inplace=True)
analyte_preferred_count

,count
immunology_symbol,
AFP,259
CCL22,636
CSF3,470
CTACK,72
CXCL9,28
Eotaxin,600
GMCSF,1595
GRO,248
IFNA,6802


## Number of Results for Each Study and Analyte

In [18]:
#symbol_by_study = elisa_result.groupby('study_accession').immunology_symbol.value_counts()
symbol_by_study = elisa_result.sort_values(['study_accession','immunology_symbol']).groupby(['study_accession','immunology_symbol']).immunology_symbol.count()
symbol_by_study = symbol_by_study.to_frame()
symbol_by_study.rename(columns={'immunology_symbol':'count'}, inplace=True)

In [19]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [20]:
symbol_by_study

count
study_accession immunology_symbol       
SDY208          IFNG                   4
                IL4                    4
                IgA1                   1
                TNFA                   4
SDY21           CSF3                 216
                Eotaxin              179
                GMCSF                177
                GRO                  220
                IFNG                 180
                IL10                 194
                IL12p40              213
                IL13                 190
                IL17                 116
                IL1A                 215
                IL1B                 237
                IL2                  188
                IL3                  174
                IL4                  157
                IL5                  121
                IL6                  175
                IL9                  212
                MCP1                 227
                MIP1A                179
                MIP1B                224
                RANTES               232
                TNFA                 206
SDY224          IgA1                 141
                IgM                  141
SDY269          Eotaxin              167
                IFNA2                167
                IFNG                 167
                IL12p70              167
                IL1A                 167
                IL1B                 167
                IP-10                167
                MCP1                 167
                MIP1A                167
                RANTES               167
SDY28           IFNA                6211
                IFNB1               5255
                IFNG                6204
                IL10                5998
                IL12p40             5342
                IL12p70             5324
                IL18                5196
                IL1B                5518
                IL2                 5201
                IL4                 5980
                IL6                 5056
                TNFA                5506
SDY283          IFNA2                774
                IFNG                 774
                IL10                 774
                IL12p40              774
                IL12p70              774
                IL1B                 716
                IL23A                602
                IL6                  737
                TNFA                 761
SDY29           IgA1                 456
                IgM                  600
SDY36           AFP                  259
                CSF3                 226
                Eotaxin              226
                GMCSF                725
                IFNA                 521
                IFNA2               1005
                IFNG                1246
                IGF1                 179
                IGFBP3               179
                IL10                 747
                IL12p40             1190
                IL12p70             1190
                IL13                1246
                IL15                1190
                IL17                 226
                IL1A                1190
                IL1B                1190
                IL2                  747
                IL3                  226
                IL4                  747
                IL6                 2210
                IL7                  226
                IL8                 2210
                IP-10               1190
                LEP                  179
                MCP1                 226
                MIP1A                655
                MIP1B                569
                RANTES               226
                TNFA                2210
                VEGFA                405
SDY396          IgA1                 214
                IgM                  214
SDY40           IL8                 3624
                MIF                 2833
                T

## Study - Analyte - Count

In [21]:
#data= []
#studies = sorted(elisa_result['study_accession'].unique())
#for study in studies:
#    study_rows = elisa_result[elisa_result['study_accession']==study]
#    analytes = sorted(study_rows['immunology_symbol'].unique())
#    for analyte in analytes:
#        values = study_rows[study_rows['immunology_symbol']==analyte]
#        data.append([study, analyte, len(values)])
#
#display(HTML(
#    '<table><tr><th>Study</th><th>Analyte</th><th>Count</th></tr><tr>{}</tr></table>'.format(
#        '</tr><tr>'.join(
#           '<td>{}</td>'.format('</td><td>'.join(str(_) for _ in row)) for row in data)
#        )
#))


## Study - Analyte - Planned Visit - Arm Descriptive Statistics

In [22]:
InteractiveShell.ast_node_interactivity = "all"
pd.options.display.float_format = '{:.2f}'.format

In [23]:
data= []
studies = sorted(elisa_result['study_accession'].unique())
for study in studies:
    study_rows = elisa_result[elisa_result['study_accession']==study]
    analytes = sorted(study_rows['immunology_symbol'].unique())
    #print(study)
    #study_rows.groupby('immunology_symbol')['value_preferred'].describe()
    study_rows.groupby(['study_accession','immunology_symbol','planned_visit_accession','arm_accession'])['value_preferred'].describe()
#    for analyte in analytes:
#        values = study_rows[study_rows['immunology_symbol']==analyte]
#        print(values['value_preferred'].describe())

count  \
study_accession immunology_symbol planned_visit_accession arm_accession          
SDY208          IFNG              PV2442                  ARM884          1.00   
                                                          ARM885          1.00   
                                                          ARM886          1.00   
                                                          ARM887          1.00   
                IL4               PV2442                  ARM884          1.00   
                                                          ARM885          1.00   
                                                          ARM886          1.00   
                                                          ARM887          1.00   
                IgA1              PV2442                  ARM884          1.00   
                TNFA              PV2442                  ARM884          1.00   
                                                          ARM885          1.00   
                                                          ARM886          1.00   
                                                          ARM887          1.00   

                                                                            mean  \
study_accession immunology_symbol planned_visit_accession arm_accession            
SDY208          IFNG              PV2442                  ARM884          860.00   
                                                          ARM885          392.00   
                                                          ARM886          321.00   
                                                          ARM887           71.00   
                IL4               PV2442                  ARM884          198.50   
                                                          ARM885          280.00   
                                                          ARM886           90.00   
                                                          ARM887           55.00   
                IgA1              PV2442                  ARM884        65000.00   
                TNFA              PV2442                  ARM884           99.00   
                                                          ARM885           97.00   
                                                          ARM886          185.00   
                                                          ARM887           16.00   

                                                                         std  \
study_accession immunology_symbol planned_visit_accession arm_accession        
SDY208          IFNG              PV2442                  ARM884         nan   
                                                          ARM885         nan   
                                                          ARM886         nan   
                                                          ARM887         nan   
                IL4               PV2442                  ARM884         nan   
                                                          ARM885         nan   
                                                          ARM886         nan   
                                                          ARM887         nan   
                IgA1              PV2442                  ARM884         nan   
                TNFA              PV2442                  ARM884         nan   
                                                          ARM885         nan   
                                                          ARM886         nan   
                                                          ARM887         nan   

                                                                             min  \
study_accession immunology_symbol planned_visit_accession arm_accession            
SDY208          IFNG              PV2442                  ARM884          860.00   
                                                          ARM885          392.00   
                                            

count  \
study_accession immunology_symbol planned_visit_accession arm_accession          
SDY21           CSF3              PV2911                  ARM316         94.00   
                                                          ARM317         98.00   
                                  PV2912                  ARM316         20.00   
                                                          ARM317          4.00   
                Eotaxin           PV2911                  ARM316         68.00   
                                                          ARM317         91.00   
                                  PV2912                  ARM316         16.00   
                                                          ARM317          4.00   
                GMCSF             PV2911                  ARM316         57.00   
                                                          ARM317         98.00   
                                  PV2912                  ARM316         18.00   
                                                          ARM317          4.00   
                GRO               PV2911                  ARM316        116.00   
                                                          ARM317         82.00   
                                  PV2912                  ARM316         20.00   
                                                          ARM317          2.00   
                IFNG              PV2911                  ARM316         70.00   
                                                          ARM317         91.00   
                                  PV2912                  ARM316         17.00   
                                                          ARM317          2.00   
                IL10              PV2911                  ARM316         74.00   
                                                          ARM317         96.00   
                                  PV2912                  ARM316         20.00   
                                                          ARM317          4.00   
                IL12p40           PV2911                  ARM316        121.00   
                                                          ARM317         70.00   
                                  PV2912                  ARM316         20.00   
                                                          ARM317          2.00   
                IL13              PV2911                  ARM316         68.00   
                                                          ARM317         98.00   
                                  PV2912                  ARM316         20.00   
                                                          ARM317          4.00   
                IL17              PV2911                  ARM316         30.00   
                                                          ARM317         69.00   
                                  PV2912                  ARM316         15.00   
                                                          ARM317          2.00   
                IL1A              PV2911                  ARM316         93.00   
                                                          ARM317         98.00   
                                  PV2912                  ARM316         20.00   
                                                          ARM317          4.00   
                IL1B              PV2911                  ARM316        115.00   
                                                          ARM317         98.00   
                                  PV2912                  ARM316         20.00   
                                                          ARM317          4.00   
                IL2               PV2911                  ARM316         86.00   
                                                          ARM317         80.00   
                                  PV2912                  ARM316         20.00   
                                                       

count  \
study_accession immunology_symbol planned_visit_accession arm_accession          
SDY224          IgA1              PV2550                  ARM926         13.00   
                                  PV2551                  ARM926         13.00   
                                  PV2552                  ARM926         13.00   
                                  PV2553                  ARM926         13.00   
                                  PV2554                  ARM926         12.00   
                                  PV2555                  ARM926         13.00   
                                  PV2556                  ARM926         13.00   
                                  PV2557                  ARM926         13.00   
                                  PV2558                  ARM926         13.00   
                                  PV2559                  ARM926         13.00   
                                  PV2560                  ARM926         12.00   
                IgM               PV2550                  ARM926         13.00   
                                  PV2551                  ARM926         13.00   
                                  PV2552                  ARM926         13.00   
                                  PV2553                  ARM926         13.00   
                                  PV2554                  ARM926         12.00   
                                  PV2555                  ARM926         13.00   
                                  PV2556                  ARM926         13.00   
                                  PV2557                  ARM926         13.00   
                                  PV2558                  ARM926         13.00   
                                  PV2559                  ARM926         13.00   
                                  PV2560                  ARM926         12.00   

                                                                               mean  \
study_accession immunology_symbol planned_visit_accession arm_accession               
SDY224          IgA1              PV2550                  ARM926         9110546.15   
                                  PV2551                  ARM926         8900497.69   
                                  PV2552                  ARM926         8805404.62   
                                  PV2553                  ARM926         8919989.23   
                                  PV2554                  ARM926         8925630.83   
                                  PV2555                  ARM926         9092790.77   
                                  PV2556                  ARM926        10633471.54   
                                  PV2557                  ARM926        12472708.46   
                                  PV2558                  ARM926        18337014.62   
                                  PV2559                  ARM926        18566039.23   
                                  PV2560                  ARM926        18066185.83   
                IgM               PV2550                  ARM926         6694241.54   
                                  PV2551                  ARM926         6432882.00   
                                  PV2552                  ARM926         6412168.62   
                                  PV2553                  ARM926         6491837.92   
                                  PV2554                  ARM926         7328060.17   
                                  PV2555                  ARM926        13050051.38   
                                  PV2556                  ARM926        21675168.46   
                                  PV2557                  ARM926        26347896.15   
                                  PV2558                  ARM926        27616872.31   
                                  PV2559                  ARM926        28084817.69   
                                  PV2560                  ARM926        26628904.17   

               

count  \
study_accession immunology_symbol planned_visit_accession arm_accession          
SDY269          Eotaxin           PV2723                  ARM1888        28.00   
                                                          ARM1889        27.00   
                                  PV2724                  ARM1888        28.00   
                                                          ARM1889        27.00   
                                  PV2725                  ARM1888        28.00   
                                                          ARM1889        29.00   
                IFNA2             PV2723                  ARM1888        28.00   
                                                          ARM1889        27.00   
                                  PV2724                  ARM1888        28.00   
                                                          ARM1889        27.00   
                                  PV2725                  ARM1888        28.00   
                                                          ARM1889        29.00   
                IFNG              PV2723                  ARM1888        28.00   
                                                          ARM1889        27.00   
                                  PV2724                  ARM1888        28.00   
                                                          ARM1889        27.00   
                                  PV2725                  ARM1888        28.00   
                                                          ARM1889        29.00   
                IL12p70           PV2723                  ARM1888        28.00   
                                                          ARM1889        27.00   
                                  PV2724                  ARM1888        28.00   
                                                          ARM1889        27.00   
                                  PV2725                  ARM1888        28.00   
                                                          ARM1889        29.00   
                IL1A              PV2723                  ARM1888        28.00   
                                                          ARM1889        27.00   
                                  PV2724                  ARM1888        28.00   
                                                          ARM1889        27.00   
                                  PV2725                  ARM1888        28.00   
                                                          ARM1889        29.00   
                IL1B              PV2723                  ARM1888        28.00   
                                                          ARM1889        27.00   
                                  PV2724                  ARM1888        28.00   
                                                          ARM1889        27.00   
                                  PV2725                  ARM1888        28.00   
                                                          ARM1889        29.00   
                IP-10             PV2723                  ARM1888        28.00   
                                                          ARM1889        27.00   
                                  PV2724                  ARM1888        28.00   
                                                          ARM1889        27.00   
                                  PV2725                  ARM1888        28.00   
                                                          ARM1889        29.00   
                MCP1              PV2723                  ARM1888        28.00   
                                                          ARM1889        27.00   
                                  PV2724                  ARM1888        28.00   
                                                          ARM1889        27.00   
                                  PV2725                  ARM1888        28.00   
                                                       

count  \
study_accession immunology_symbol planned_visit_accession arm_accession           
SDY28           IFNA              PV1262                  ARM81         6211.00   
                IFNB1             PV1262                  ARM81         5255.00   
                IFNG              PV1262                  ARM81         6204.00   
                IL10              PV1262                  ARM81         5998.00   
                IL12p40           PV1262                  ARM81         5342.00   
                IL12p70           PV1262                  ARM81         5324.00   
                IL18              PV1262                  ARM81         5196.00   
                IL1B              PV1262                  ARM81         5518.00   
                IL2               PV1262                  ARM81         5201.00   
                IL4               PV1262                  ARM81         5980.00   
                IL6               PV1262                  ARM81         5056.00   
                TNFA              PV1262                  ARM81         5506.00   

                                                                           mean  \
study_accession immunology_symbol planned_visit_accession arm_accession           
SDY28           IFNA              PV1262                  ARM81           39.37   
                IFNB1             PV1262                  ARM81          -14.52   
                IFNG              PV1262                  ARM81         1307.71   
                IL10              PV1262                  ARM81           24.01   
                IL12p40           PV1262                  ARM81          115.46   
                IL12p70           PV1262                  ARM81            0.17   
                IL18              PV1262                  ARM81           -1.11   
                IL1B              PV1262                  ARM81           75.26   
                IL2               PV1262                  ARM81           33.32   
                IL4               PV1262                  ARM81            7.55   
                IL6               PV1262                  ARM81         1311.53   
                TNFA              PV1262                  ARM81          281.60   

                                                                            std  \
study_accession immunology_symbol planned_visit_accession arm_accession           
SDY28           IFNA              PV1262                  ARM81           80.56   
                IFNB1             PV1262                  ARM81           33.08   
                IFNG              PV1262                  ARM81         2227.76   
                IL10              PV1262                  ARM81           38.79   
                IL12p40           PV1262                  ARM81          210.35   
                IL12p70           PV1262                  ARM81           20.51   
                IL18              PV1262                  ARM81           27.37   
                IL1B              PV1262                  ARM81          186.02   
                IL2               PV1262                  ARM81           47.44   
                IL4               PV1262                  ARM81           16.63   
                IL6               PV1262                  ARM81         1313.56   
                TNFA              PV1262                  ARM81          320.38   

                                                                            min  \
study_accession immunology_symbol planned_visit_accession arm_accession           
SDY28           IFNA              PV1262                  ARM81          -92.47   
                IFNB1             PV1262                  ARM81         -171.61   
                IFNG              PV1262                  ARM81         -276.74   
                IL10              PV1262                  ARM81           -8.46   
                IL12p40           PV1262                  ARM81         -115.69   
    

...

count  \
study_accession immunology_symbol planned_visit_accession arm_accession          
SDY739          IgA1              PV4682                  ARM3059        56.00   
                                                          ARM3060        70.00   
                                  PV4683                  ARM3059        56.00   
                                                          ARM3060        70.00   
                                  PV4684                  ARM3059        56.00   
                                                          ARM3060        70.00   
                IgM               PV4682                  ARM3059        56.00   
                                                          ARM3060        70.00   
                                  PV4683                  ARM3059        56.00   
                                                          ARM3060        70.00   
                                  PV4684                  ARM3059        56.00   
                                                          ARM3060        70.00   

                                                                                mean  \
study_accession immunology_symbol planned_visit_accession arm_accession                
SDY739          IgA1              PV4682                  ARM3059        17335668.39   
                                                          ARM3060        26294701.44   
                                  PV4683                  ARM3059        86891030.71   
                                                          ARM3060        76846133.29   
                                  PV4684                  ARM3059        63916871.39   
                                                          ARM3060        74104816.00   
                IgM               PV4682                  ARM3059        79339788.75   
                                                          ARM3060        65117386.40   
                                  PV4683                  ARM3059       199096879.11   
                                                          ARM3060       103567043.93   
                                  PV4684                  ARM3059       131381470.36   
                                                          ARM3060       124823106.86   

                                                                                 std  \
study_accession immunology_symbol planned_visit_accession arm_accession                
SDY739          IgA1              PV4682                  ARM3059        16642319.07   
                                                          ARM3060        32395947.26   
                                  PV4683                  ARM3059        85326684.22   
                                                          ARM3060        81550226.24   
                                  PV4684                  ARM3059        66689930.35   
                                                          ARM3060        87794039.72   
                IgM               PV4682                  ARM3059       111120529.54   
                                                          ARM3060        86494236.03   
                                  PV4683                  ARM3059       408668385.28   
                                                          ARM3060       165649166.01   
                                  PV4684                  ARM3059       238364685.66   
                                                          ARM3060       230407707.14   

                                                                                min  \
study_accession immunology_symbol planned_visit_accession arm_accession               
SDY739          IgA1              PV4682                  ARM3059       -5067700.00   
                                                          ARM3060         857511.00   
                                  PV4683                  ARM3059        1000000.00   
                       

count  \
study_accession immunology_symbol planned_visit_accession arm_accession          
SDY8            IFNA              PV1546                  ARM263          2.00   
                                                          ARM264          1.00   
                                                          ARM265          1.00   
                IL10              PV1546                  ARM263         34.00   
                                                          ARM264         27.00   
                                                          ARM265         25.00   
                IL13              PV1546                  ARM263         27.00   
                                                          ARM264         23.00   
                                                          ARM265         21.00   
                IL17              PV1546                  ARM263          2.00   
                                                          ARM264          1.00   
                                                          ARM265          5.00   
                TNFA              PV1546                  ARM263         30.00   
                                                          ARM264         22.00   
                                                          ARM265         30.00   

                                                                          mean  \
study_accession immunology_symbol planned_visit_accession arm_accession          
SDY8            IFNA              PV1546                  ARM263         28.80   
                                                          ARM264         15.30   
                                                          ARM265         15.50   
                IL10              PV1546                  ARM263        124.06   
                                                          ARM264        106.31   
                                                          ARM265        114.43   
                IL13              PV1546                  ARM263         57.45   
                                                          ARM264         97.36   
                                                          ARM265        107.81   
                IL17              PV1546                  ARM263         40.10   
                                                          ARM264        116.70   
                                                          ARM265        190.76   
                TNFA              PV1546                  ARM263        207.15   
                                                          ARM264         73.31   
                                                          ARM265        483.92   

                                                                           std  \
study_accession immunology_symbol planned_visit_accession arm_accession          
SDY8            IFNA              PV1546                  ARM263         16.97   
                                                          ARM264           nan   
                                                          ARM265           nan   
                IL10              PV1546                  ARM263         93.68   
                                                          ARM264         63.85   
                                                          ARM265         84.52   
                IL13              PV1546                  ARM263         43.02   
                                                          ARM264        121.88   
                                                          ARM265         93.36   
                IL17              PV1546                  ARM263          6.22   
                                                          ARM264           nan   
                                                          ARM265        189.47   
                TNFA              PV1546                  ARM263        300.37   
                                                     

count  \
study_accession immunology_symbol planned_visit_accession arm_accession          
SDY819          IgA1              PV4963                  ARM3203        30.00   
                                                          ARM3204        60.00   
                                  PV4964                  ARM3203        30.00   
                                                          ARM3204        60.00   
                                  PV4965                  ARM3203        30.00   
                                                          ARM3204        60.00   
                IgM               PV4963                  ARM3203        30.00   
                                                          ARM3204        60.00   
                                  PV4964                  ARM3203        30.00   
                                                          ARM3204        60.00   
                                  PV4965                  ARM3203        30.00   
                                                          ARM3204        60.00   

                                                                                mean  \
study_accession immunology_symbol planned_visit_accession arm_accession                
SDY819          IgA1              PV4963                  ARM3203         7392065.00   
                                                          ARM3204        12020248.83   
                                  PV4964                  ARM3203        15684396.67   
                                                          ARM3204        14093367.00   
                                  PV4965                  ARM3203        13309398.33   
                                                          ARM3204        15961554.50   
                IgM               PV4963                  ARM3203        49626313.67   
                                                          ARM3204        54866287.17   
                                  PV4964                  ARM3203       153251986.67   
                                                          ARM3204        84288795.33   
                                  PV4965                  ARM3203       142264100.00   
                                                          ARM3204        91219803.00   

                                                                                 std  \
study_accession immunology_symbol planned_visit_accession arm_accession                
SDY819          IgA1              PV4963                  ARM3203         4246329.11   
                                                          ARM3204         9016836.48   
                                  PV4964                  ARM3203         9107441.90   
                                                          ARM3204        10843521.97   
                                  PV4965                  ARM3203         8702170.76   
                                                          ARM3204        13263413.31   
                IgM               PV4963                  ARM3203        34614943.07   
                                                          ARM3204        42915954.58   
                                  PV4964                  ARM3203       107247757.07   
                                                          ARM3204        82779251.11   
                                  PV4965                  ARM3203       107995467.98   
                                                          ARM3204        80974704.63   

                                                                                min  \
study_accession immunology_symbol planned_visit_accession arm_accession               
SDY819          IgA1              PV4963                  ARM3203        1132060.00   
                                                          ARM3204        1893380.00   
                                  PV4964                  ARM3203        3584550.00   
                       

count  \
study_accession immunology_symbol planned_visit_accession arm_accession          
SDY9            IL17              PV1548                  ARM266         20.00   
                                                          ARM267         19.00   
                                                          ARM268         25.00   

                                                                         mean  \
study_accession immunology_symbol planned_visit_accession arm_accession         
SDY9            IL17              PV1548                  ARM266        17.18   
                                                          ARM267        10.97   
                                                          ARM268        66.42   

                                                                           std  \
study_accession immunology_symbol planned_visit_accession arm_accession          
SDY9            IL17              PV1548                  ARM266         31.51   
                                                          ARM267         23.54   
                                                          ARM268        203.71   

                                                                         min  \
study_accession immunology_symbol planned_visit_accession arm_accession        
SDY9            IL17              PV1548                  ARM266        0.01   
                                                          ARM267        0.01   
                                                          ARM268        0.01   

                                                                         25%  \
study_accession immunology_symbol planned_visit_accession arm_accession        
SDY9            IL17              PV1548                  ARM266        0.01   
                                                          ARM267        0.01   
                                                          ARM268        0.01   

                                                                         50%  \
study_accession immunology_symbol planned_visit_accession arm_accession        
SDY9            IL17              PV1548                  ARM266        0.01   
                                                          ARM267        0.01   
                                                          ARM268        0.01   

                                                                          75%  \
study_accession immunology_symbol planned_visit_accession arm_accession         
SDY9            IL17              PV1548                  ARM266        27.71   
                                                          ARM267         0.01   
                                                          ARM268        27.01   

                                                                           max  
study_accession immunology_symbol planned_visit_accession arm_accession         
SDY9            IL17              PV1548                  ARM266        104.37  
                                                          ARM267         80.25  
                                                          ARM268        896.04

count  \
study_accession immunology_symbol planned_visit_accession arm_accession          
SDY95           CSF3              PV2819                  ARM590          5.00   
                                                          ARM592          5.00   
                                                          ARM596          4.00   
                                                          ARM599          5.00   
                                                          ARM601          5.00   
                                                          ARM605          4.00   
                CXCL9             PV2819                  ARM590          5.00   
                                                          ARM592          5.00   
                                                          ARM596          4.00   
                                                          ARM599          5.00   
                                                          ARM601          5.00   
                                                          ARM605          4.00   
                Eotaxin           PV2819                  ARM590          5.00   
                                                          ARM592          5.00   
                                                          ARM596          4.00   
                                                          ARM599          5.00   
                                                          ARM601          5.00   
                                                          ARM605          4.00   
                GMCSF             PV2819                  ARM590          5.00   
                                                          ARM592          5.00   
                                                          ARM596          4.00   
                                                          ARM599          5.00   
                                                          ARM601          5.00   
                                                          ARM605          4.00   
                GRO               PV2819                  ARM590          5.00   
                                                          ARM592          5.00   
                                                          ARM596          4.00   
                                                          ARM599          5.00   
                                                          ARM601          5.00   
                                                          ARM605          4.00   
                IFNA              PV2819                  ARM590          5.00   
                                                          ARM592          5.00   
                                                          ARM596          4.00   
                                                          ARM599          5.00   
                                                          ARM601          5.00   
                                                          ARM605          4.00   
                IFNG              PV2819                  ARM590          5.00   
                                                          ARM592          5.00   
                                                          ARM596          4.00   
                                                          ARM599          5.00   
                                                          ARM601          5.00   
                                                          ARM605          4.00   
                IL10              PV2819                  ARM590          5.00   
                                                          ARM592          5.00   
                                                          ARM596          4.00   
                                                          ARM599          5.00   
                                                          ARM601          5.00   
                                                       

In [24]:
data= []
analytes = sorted(elisa_result['immunology_symbol'].unique())
for analyte in analytes:
    analyte_rows = elisa_result[elisa_result['immunology_symbol']==analyte]
    studies = sorted(analyte_rows['study_accession'].unique())
    analyte_rows.groupby(['immunology_symbol','study_accession','planned_visit_accession','arm_accession'])['value_preferred'].describe()
#    for analyte in analytes:
#        values = study_rows[study_rows['immunology_symbol']==analyte]
#        print(values['value_preferred'].describe())

count  \
immunology_symbol study_accession planned_visit_accession arm_accession          
AFP               SDY36           PV1476                  ARM210        150.00   
                                  PV1477                  ARM210         39.00   
                                  PV1478                  ARM210         20.00   
                                  PV1479                  ARM210          4.00   
                                  PV1480                  ARM210         19.00   
                                  PV1481                  ARM210         27.00   

                                                                             mean  \
immunology_symbol study_accession planned_visit_accession arm_accession             
AFP               SDY36           PV1476                  ARM210        116156.09   
                                  PV1477                  ARM210         90858.97   
                                  PV1478                  ARM210          5417.00   
                                  PV1479                  ARM210          3892.50   
                                  PV1480                  ARM210          3079.47   
                                  PV1481                  ARM210          3034.44   

                                                                              std  \
immunology_symbol study_accession planned_visit_accession arm_accession             
AFP               SDY36           PV1476                  ARM210         92286.00   
                                  PV1477                  ARM210        134738.98   
                                  PV1478                  ARM210         11317.03   
                                  PV1479                  ARM210          1206.71   
                                  PV1480                  ARM210          1458.84   
                                  PV1481                  ARM210          4896.40   

                                                                             min  \
immunology_symbol study_accession planned_visit_accession arm_accession            
AFP               SDY36           PV1476                  ARM210        -2000.00   
                                  PV1477                  ARM210         1020.00   
                                  PV1478                  ARM210          930.00   
                                  PV1479                  ARM210         2530.00   
                                  PV1480                  ARM210          810.00   
                                  PV1481                  ARM210        -2000.00   

                                                                             25%  \
immunology_symbol study_accession planned_visit_accession arm_accession            
AFP               SDY36           PV1476                  ARM210        34887.50   
                                  PV1477                  ARM210         2780.00   
                                  PV1478                  ARM210         1582.50   
                                  PV1479                  ARM210         3460.00   
                                  PV1480                  ARM210         1900.00   
                                  PV1481                  ARM210         1320.00   

                                                                              50%  \
immunology_symbol study_accession planned_visit_accession arm_accession             
AFP               SDY36           PV1476                  ARM210        105215.00   
                                  PV1477                  ARM210          4270.00   
                                  PV1478                  ARM210          3070.00   
                                  PV1479                  ARM210          3785.00   
                                  PV1480                  ARM210          3140.00   
                                  PV1481                  ARM210          1840.00   

                           

count  \
immunology_symbol study_accession planned_visit_accession arm_accession          
CCL22             SDY67           PV1792                  ARM544        159.00   
                                  PV1793                  ARM544        159.00   
                                  PV1794                  ARM544        159.00   
                                  PV1795                  ARM544        159.00   

                                                                          mean  \
immunology_symbol study_accession planned_visit_accession arm_accession          
CCL22             SDY67           PV1792                  ARM544        382.23   
                                  PV1793                  ARM544        360.87   
                                  PV1794                  ARM544        377.65   
                                  PV1795                  ARM544        388.62   

                                                                           std  \
immunology_symbol study_accession planned_visit_accession arm_accession          
CCL22             SDY67           PV1792                  ARM544        406.83   
                                  PV1793                  ARM544        375.88   
                                  PV1794                  ARM544        411.15   
                                  PV1795                  ARM544        470.49   

                                                                          min  \
immunology_symbol study_accession planned_visit_accession arm_accession         
CCL22             SDY67           PV1792                  ARM544        80.77   
                                  PV1793                  ARM544        74.74   
                                  PV1794                  ARM544        87.73   
                                  PV1795                  ARM544         3.14   

                                                                           25%  \
immunology_symbol study_accession planned_visit_accession arm_accession          
CCL22             SDY67           PV1792                  ARM544        190.40   
                                  PV1793                  ARM544        180.90   
                                  PV1794                  ARM544        184.00   
                                  PV1795                  ARM544        188.80   

                                                                           50%  \
immunology_symbol study_accession planned_visit_accession arm_accession          
CCL22             SDY67           PV1792                  ARM544        253.95   
                                  PV1793                  ARM544        251.13   
                                  PV1794                  ARM544        250.88   
                                  PV1795                  ARM544        255.10   

                                                                           75%  \
immunology_symbol study_accession planned_visit_accession arm_accession          
CCL22             SDY67           PV1792                  ARM544        375.06   
                                  PV1793                  ARM544        346.12   
                                  PV1794                  ARM544        374.91   
                                  PV1795                  ARM544        365.73   

                                                                            max  
immunology_symbol study_accession planned_visit_accession arm_accession          
CCL22             SDY67           PV1792                  ARM544        2445.37  
                                  PV1793                  ARM544        2299.50  
                                  PV1794                  ARM544        2928.61  
                                  PV1795                  ARM544        3801.62

count  \
immunology_symbol study_accession planned_visit_accession arm_accession          
CSF3              SDY21           PV2911                  ARM316         94.00   
                                                          ARM317         98.00   
                                  PV2912                  ARM316         20.00   
                                                          ARM317          4.00   
                  SDY36           PV1476                  ARM210        138.00   
                                  PV1477                  ARM210         33.00   
                                  PV1478                  ARM210         15.00   
                                  PV1479                  ARM210          4.00   
                                  PV1480                  ARM210         17.00   
                                  PV1481                  ARM210         19.00   
                  SDY95           PV2819                  ARM590          5.00   
                                                          ARM592          5.00   
                                                          ARM596          4.00   
                                                          ARM599          5.00   
                                                          ARM601          5.00   
                                                          ARM605          4.00   

                                                                            mean  \
immunology_symbol study_accession planned_visit_accession arm_accession            
CSF3              SDY21           PV2911                  ARM316           15.33   
                                                          ARM317         1512.77   
                                  PV2912                  ARM316          101.82   
                                                          ARM317        10498.94   
                  SDY36           PV1476                  ARM210           47.69   
                                  PV1477                  ARM210           36.05   
                                  PV1478                  ARM210           26.39   
                                  PV1479                  ARM210           17.44   
                                  PV1480                  ARM210           26.73   
                                  PV1481                  ARM210           29.17   
                  SDY95           PV2819                  ARM590           83.11   
                                                          ARM592          695.81   
                                                          ARM596            7.55   
                                                          ARM599           15.68   
                                                          ARM601          111.51   
                                                          ARM605           36.71   

                                                                             std  \
immunology_symbol study_accession planned_visit_accession arm_accession            
CSF3              SDY21           PV2911                  ARM316           26.54   
                                                          ARM317         3721.86   
                                  PV2912                  ARM316           77.73   
                                                          ARM317        14646.34   
                  SDY36           PV1476                  ARM210           21.41   
                                  PV1477                  ARM210           22.40   
                                  PV1478                  ARM210           12.01   
                                  PV1479                  ARM210           10.18   
                                  PV1480                  ARM210           10.03   
                                  PV1481                  ARM210           19.60   
                  SDY95           PV2819                  ARM590           

count  \
immunology_symbol study_accession planned_visit_accession arm_accession          
CTACK             SDY6            PV1538                  ARM251         36.00   
                                                          ARM252         35.00   
                                                          ARM253          1.00   

                                                                           mean  \
immunology_symbol study_accession planned_visit_accession arm_accession           
CTACK             SDY6            PV1538                  ARM251        1204.30   
                                                          ARM252         599.29   
                                                          ARM253         246.80   

                                                                            std  \
immunology_symbol study_accession planned_visit_accession arm_accession           
CTACK             SDY6            PV1538                  ARM251        2232.83   
                                                          ARM252         312.38   
                                                          ARM253            nan   

                                                                           min  \
immunology_symbol study_accession planned_visit_accession arm_accession          
CTACK             SDY6            PV1538                  ARM251        232.10   
                                                          ARM252        208.80   
                                                          ARM253        246.80   

                                                                           25%  \
immunology_symbol study_accession planned_visit_accession arm_accession          
CTACK             SDY6            PV1538                  ARM251        473.15   
                                                          ARM252        384.75   
                                                          ARM253        246.80   

                                                                           50%  \
immunology_symbol study_accession planned_visit_accession arm_accession          
CTACK             SDY6            PV1538                  ARM251        646.40   
                                                          ARM252        493.40   
                                                          ARM253        246.80   

                                                                            75%  \
immunology_symbol study_accession planned_visit_accession arm_accession           
CTACK             SDY6            PV1538                  ARM251        1200.18   
                                                          ARM252         738.10   
                                                          ARM253         246.80   

                                                                             max  
immunology_symbol study_accession planned_visit_accession arm_accession           
CTACK             SDY6            PV1538                  ARM251        13851.00  
                                                          ARM252         1618.60  
                                                          ARM253          246.80

count  \
immunology_symbol study_accession planned_visit_accession arm_accession          
CXCL9             SDY95           PV2819                  ARM590          5.00   
                                                          ARM592          5.00   
                                                          ARM596          4.00   
                                                          ARM599          5.00   
                                                          ARM601          5.00   
                                                          ARM605          4.00   

                                                                            mean  \
immunology_symbol study_accession planned_visit_accession arm_accession            
CXCL9             SDY95           PV2819                  ARM590         4432.40   
                                                          ARM592        17498.05   
                                                          ARM596         1540.77   
                                                          ARM599         3388.76   
                                                          ARM601        15239.10   
                                                          ARM605         3678.91   

                                                                             std  \
immunology_symbol study_accession planned_visit_accession arm_accession            
CXCL9             SDY95           PV2819                  ARM590         3168.16   
                                                          ARM592        11802.69   
                                                          ARM596          518.55   
                                                          ARM599         2193.24   
                                                          ARM601        15276.82   
                                                          ARM605         2235.14   

                                                                            min  \
immunology_symbol study_accession planned_visit_accession arm_accession           
CXCL9             SDY95           PV2819                  ARM590        1663.91   
                                                          ARM592        1253.44   
                                                          ARM596        1069.00   
                                                          ARM599        1675.37   
                                                          ARM601        1128.36   
                                                          ARM605         896.57   

                                                                            25%  \
immunology_symbol study_accession planned_visit_accession arm_accession           
CXCL9             SDY95           PV2819                  ARM590        2046.14   
                                                          ARM592        9728.22   
                                                          ARM596        1150.50   
                                                          ARM599        1721.14   
                                                          ARM601        8791.43   
                                                          ARM605        2454.96   

                                                                             50%  \
immunology_symbol study_accession planned_visit_accession arm_accession            
CXCL9             SDY95           PV2819                  ARM590         3797.18   
                                                          ARM592        21864.30   
                                                          ARM596         1454.04   
                                                          ARM599         3107.13   
                                                          ARM601        10481.30   
                                                          ARM605         3908.80   

                                                                   

...

count  \
immunology_symbol study_accession planned_visit_accession arm_accession          
RANTES            SDY21           PV2911                  ARM316        115.00   
                                                          ARM317         97.00   
                                  PV2912                  ARM316         16.00   
                                                          ARM317          4.00   
                  SDY269          PV2723                  ARM1888        28.00   
                                                          ARM1889        27.00   
                                  PV2724                  ARM1888        28.00   
                                                          ARM1889        27.00   
                                  PV2725                  ARM1888        28.00   
                                                          ARM1889        29.00   
                  SDY36           PV1476                  ARM210        138.00   
                                  PV1477                  ARM210         33.00   
                                  PV1478                  ARM210         15.00   
                                  PV1479                  ARM210          4.00   
                                  PV1480                  ARM210         17.00   
                                  PV1481                  ARM210         19.00   
                  SDY67           PV1792                  ARM544        159.00   
                                  PV1793                  ARM544        159.00   
                                  PV1794                  ARM544        159.00   
                                  PV1795                  ARM544        159.00   
                  SDY95           PV2819                  ARM590          5.00   
                                                          ARM592          5.00   
                                                          ARM596          4.00   
                                                          ARM599          5.00   
                                                          ARM601          5.00   
                                                          ARM605          4.00   

                                                                             mean  \
immunology_symbol study_accession planned_visit_accession arm_accession             
RANTES            SDY21           PV2911                  ARM316         12171.72   
                                                          ARM317           610.07   
                                  PV2912                  ARM316        980879.21   
                                                          ARM317          2253.92   
                  SDY269          PV2723                  ARM1888         1178.33   
                                                          ARM1889          882.08   
                                  PV2724                  ARM1888          869.93   
                                                          ARM1889         1023.93   
                                  PV2725                  ARM1888          970.15   
                                                          ARM1889         1099.74   
                  SDY36           PV1476                  ARM210          6331.37   
                                  PV1477                  ARM210          6206.71   
                                  PV1478                  ARM210          6152.55   
                                  PV1479                  ARM210          6583.98   
                                  PV1480                  ARM210          6362.22   
                                  PV1481                  ARM210          5996.98   
                  SDY67           PV1792                  ARM544        126393.66   
                                  PV1793                  ARM544        121617.95   
                                  PV1794                  ARM544        1175

count  \
immunology_symbol study_accession planned_visit_accession arm_accession          
TARC              SDY6            PV1538                  ARM251         36.00   
                                                          ARM252         35.00   
                                                          ARM253          1.00   
                  SDY67           PV1792                  ARM544        159.00   
                                  PV1793                  ARM544        159.00   
                                  PV1794                  ARM544        159.00   
                                  PV1795                  ARM544        159.00   

                                                                           mean  \
immunology_symbol study_accession planned_visit_accession arm_accession           
TARC              SDY6            PV1538                  ARM251        1080.80   
                                                          ARM252         710.53   
                                                          ARM253          69.70   
                  SDY67           PV1792                  ARM544         629.38   
                                  PV1793                  ARM544         613.01   
                                  PV1794                  ARM544         669.89   
                                  PV1795                  ARM544         804.27   

                                                                            std  \
immunology_symbol study_accession planned_visit_accession arm_accession           
TARC              SDY6            PV1538                  ARM251         690.91   
                                                          ARM252         514.17   
                                                          ARM253            nan   
                  SDY67           PV1792                  ARM544         824.55   
                                  PV1793                  ARM544         839.55   
                                  PV1794                  ARM544        1024.01   
                                  PV1795                  ARM544        1972.89   

                                                                           min  \
immunology_symbol study_accession planned_visit_accession arm_accession          
TARC              SDY6            PV1538                  ARM251        117.40   
                                                          ARM252         82.80   
                                                          ARM253         69.70   
                  SDY67           PV1792                  ARM544         52.09   
                                  PV1793                  ARM544         59.18   
                                  PV1794                  ARM544         53.21   
                                  PV1795                  ARM544          1.55   

                                                                           25%  \
immunology_symbol study_accession planned_visit_accession arm_accession          
TARC              SDY6            PV1538                  ARM251        487.72   
                                                          ARM252        346.10   
                                                          ARM253         69.70   
                  SDY67           PV1792                  ARM544        264.00   
                                  PV1793                  ARM544        252.19   
                                  PV1794                  ARM544        249.51   
                                  PV1795                  ARM544        289.19   

                                                                           50%  \
immunology_symbol study_accession planned_visit_accession arm_accession          
TARC              SDY6            PV1538                  ARM251        842.65   
                                                          ARM252        547.70   
                                

count  \
immunology_symbol study_accession planned_visit_accession arm_accession           
TNFA              SDY208          PV2442                  ARM884           1.00   
                                                          ARM885           1.00   
                                                          ARM886           1.00   
                                                          ARM887           1.00   
                  SDY21           PV2911                  ARM316          97.00   
                                                          ARM317          85.00   
                                  PV2912                  ARM316          20.00   
                                                          ARM317           4.00   
                  SDY28           PV1262                  ARM81         5506.00   
                  SDY283          PV2833                  ARM1910        210.00   
                                                          ARM1911        126.00   
                                  PV2834                  ARM1910        227.00   
                                  PV2835                  ARM1910        198.00   
                  SDY36           PV1476                  ARM210        1400.00   
                                  PV1477                  ARM210         281.00   
                                  PV1478                  ARM210         123.00   
                                  PV1479                  ARM210          31.00   
                                  PV1480                  ARM210         157.00   
                                  PV1481                  ARM210         218.00   
                  SDY40           PV1572                  ARM303        1128.00   
                                                          ARM304        1590.00   
                                                          ARM305         912.00   
                  SDY58           PV1761                  ARM447          18.00   
                                                          ARM448          19.00   
                                                          ARM449          10.00   
                                                          ARM450           2.00   
                                                          ARM451          20.00   
                  SDY67           PV1792                  ARM544         149.00   
                                  PV1793                  ARM544         149.00   
                                  PV1794                  ARM544         149.00   
                                  PV1795                  ARM544         149.00   
                  SDY8            PV1546                  ARM263          30.00   
                                                          ARM264          22.00   
                                                          ARM265          30.00   
                  SDY95           PV2819                  ARM590           5.00   
                                                          ARM592           5.00   
                                                          ARM596           4.00   
                                                          ARM599           5.00   
                                                          ARM601           5.00   
                                                          ARM605           4.00   

                                                                            mean  \
immunology_symbol study_accession planned_visit_accession arm_accession            
TNFA              SDY208          PV2442                  ARM884           99.00   
                                                          ARM885           97.00   
                                                          ARM886          185.00   
                                                          ARM887           16.00   
                  SDY21           PV2911                  ARM316           11.70   

count  \
immunology_symbol study_accession planned_visit_accession arm_accession          
TNFRSF18          SDY58           PV1761                  ARM447         18.00   
                                                          ARM448         19.00   
                                                          ARM449         10.00   
                                                          ARM450          2.00   
                                                          ARM451         20.00   

                                                                         mean  \
immunology_symbol study_accession planned_visit_accession arm_accession         
TNFRSF18          SDY58           PV1761                  ARM447         1.11   
                                                          ARM448         5.81   
                                                          ARM449        15.49   
                                                          ARM450         4.65   
                                                          ARM451        10.00   

                                                                          std  \
immunology_symbol study_accession planned_visit_accession arm_accession         
TNFRSF18          SDY58           PV1761                  ARM447         0.95   
                                                          ARM448         9.26   
                                                          ARM449        33.85   
                                                          ARM450         2.70   
                                                          ARM451        22.99   

                                                                         min  \
immunology_symbol study_accession planned_visit_accession arm_accession        
TNFRSF18          SDY58           PV1761                  ARM447        0.17   
                                                          ARM448        0.31   
                                                          ARM449        0.45   
                                                          ARM450        2.74   
                                                          ARM451        0.19   

                                                                         25%  \
immunology_symbol study_accession planned_visit_accession arm_accession        
TNFRSF18          SDY58           PV1761                  ARM447        0.45   
                                                          ARM448        0.65   
                                                          ARM449        1.18   
                                                          ARM450        3.70   
                                                          ARM451        0.92   

                                                                         50%  \
immunology_symbol study_accession planned_visit_accession arm_accession        
TNFRSF18          SDY58           PV1761                  ARM447        0.91   
                                                          ARM448        1.26   
                                                          ARM449        2.35   
                                                          ARM450        4.65   
                                                          ARM451        3.60   

                                                                          75%  \
immunology_symbol study_accession planned_visit_accession arm_accession         
TNFRSF18          SDY58           PV1761                  ARM447         1.45   
                                                          ARM448         7.98   
                                                          ARM449        12.06   
                                                          ARM450         5.61   
                                                          ARM451         8.03   

                                                                           max  
immunology_symbol study_access

count  \
immunology_symbol study_accession planned_visit_accession arm_accession          
VEGFA             SDY36           PV1476                  ARM210        251.00   
                                  PV1477                  ARM210         63.00   
                                  PV1478                  ARM210         23.00   
                                  PV1479                  ARM210          7.00   
                                  PV1480                  ARM210         29.00   
                                  PV1481                  ARM210         32.00   
                  SDY95           PV2819                  ARM590          5.00   
                                                          ARM592          5.00   
                                                          ARM596          4.00   
                                                          ARM599          5.00   
                                                          ARM601          5.00   
                                                          ARM605          4.00   

                                                                           mean  \
immunology_symbol study_accession planned_visit_accession arm_accession           
VEGFA             SDY36           PV1476                  ARM210          94.68   
                                  PV1477                  ARM210         130.62   
                                  PV1478                  ARM210         260.19   
                                  PV1479                  ARM210         113.91   
                                  PV1480                  ARM210         281.47   
                                  PV1481                  ARM210         164.92   
                  SDY95           PV2819                  ARM590        1690.30   
                                                          ARM592        1822.19   
                                                          ARM596        1258.77   
                                                          ARM599        1223.71   
                                                          ARM601        1186.78   
                                                          ARM605        1459.03   

                                                                           std  \
immunology_symbol study_accession planned_visit_accession arm_accession          
VEGFA             SDY36           PV1476                  ARM210        260.20   
                                  PV1477                  ARM210        289.35   
                                  PV1478                  ARM210        686.01   
                                  PV1479                  ARM210        143.60   
                                  PV1480                  ARM210        515.52   
                                  PV1481                  ARM210        332.49   
                  SDY95           PV2819                  ARM590        553.37   
                                                          ARM592        833.59   
                                                          ARM596        700.79   
                                                          ARM599        460.89   
                                                          ARM601        631.88   
                                                          ARM605        535.29   

                                                                           min  \
immunology_symbol study_accession planned_visit_accession arm_accession          
VEGFA             SDY36           PV1476                  ARM210         -2.00   
                                  PV1477                  ARM210         -2.00   
                                  PV1478                  ARM210         -2.00   
                                  PV1479                  ARM210         -2.00   
                                  PV1480                  ARM210         -2.00   
                                  PV14

# Plotting

In [25]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "last_expr"
if GEN_PLOTS:
    data= []
    studies = sorted(elisa_result['study_accession'].unique())
    for study in studies:
        study_rows = elisa_result[elisa_result['study_accession']==study]
        analytes = sorted(study_rows['immunology_symbol'].unique())
        for analyte in analytes:
            analyte_rows = study_rows[study_rows['immunology_symbol']==analyte]
            #print(analyte_rows.boxplot('value_preferred', by='arm_accession'))
            fig, ax = plt.subplots(1,sharex=True)
            fig.set_size_inches(20,8)
            sns.boxplot(x='planned_visit_accession', y='value_preferred',
                    data=analyte_rows, hue='arm_accession', ax=ax);
            ax.set_title(study + ": " + analyte)
            ax.set_xlabel("")
            plt.show();
            plt.close(fig);

In [26]:
def outliers(data):
    (q25,q50,q75) = np.percentile(data, [25, 50, 75])
    iqr = q75 - q25
    mild_upper = (iqr * 1.5) + q75
    mild_lower = q25 - (iqr * 1.5)
    extreme_upper = (iqr * 3.0) + q75
    extreme_lower = q25 - (iqr * 3.0) 
    return {
        "mild_lower": mild_lower,
        "mild_upper": mild_upper,
        "extreme_lower": extreme_lower,
        "extreme_upper": extreme_upper
    }

In [27]:
def plot_value_preferred_by_study(elisa_result, study_accession, analyte, outlier):
    study_rows = elisa_result[elisa_result['study_accession']==study_accession]
    analyte_rows = study_rows[study_rows['immunology_symbol']==analyte]
    if len(analyte_rows) == 0:
        print("No Data")
        return
    checks = outliers(analyte_rows['value_preferred'])
    data_mild = analyte_rows[analyte_rows['value_preferred'].between(checks['mild_lower'],checks['mild_upper'])]
    data_extreme = analyte_rows[analyte_rows['value_preferred'].between(checks['extreme_lower'],checks['extreme_upper'])]
    if outlier == 'Mild':
        analyte_rows = data_mild
    if outlier == "Extreme":
        analyte_rows = data_extreme
    fig, ax = plt.subplots(1,sharex=True)
    fig.set_size_inches(20,8)
    sns.boxplot(x='planned_visit_accession', y='value_preferred',
        data=analyte_rows, hue='arm_accession', ax=ax);
    ax.set_title(study_accession + ": " + analyte)
    ax.set_xlabel("")
    plt.show();
    plt.close(fig);

In [28]:
def plot_value_preferred_by_analyte(elisa_result, analyte, outlier):
    analyte_rows = elisa_result[elisa_result['immunology_symbol']==analyte]
    if len(analyte_rows) == 0:
        print("No Data")
        return
    checks = outliers(analyte_rows['value_preferred'])
    data_mild = analyte_rows[analyte_rows['value_preferred'].between(checks['mild_lower'],checks['mild_upper'])]
    data_extreme = analyte_rows[analyte_rows['value_preferred'].between(checks['extreme_lower'],checks['extreme_upper'])]
    if outlier == 'Mild':
        analyte_rows = data_mild
    if outlier == "Extreme":
        analyte_rows = data_extreme
    fig, ax = plt.subplots(1,sharex=True)
    fig.set_size_inches(20,8)
    sns.boxplot(x='study_accession', y='value_preferred',
        data=analyte_rows, ax=ax);
    ax.set_title("Analyte: " + analyte)
    ax.set_xlabel("")
    plt.show();
    plt.close(fig);

In [29]:
studies = sorted(elisa_result['study_accession'].unique())
analytes_1 = sorted(elisa_result[elisa_result['study_accession']==studies[0]]['immunology_symbol'].unique())
analytes_2 = sorted(elisa_result['immunology_symbol'].unique())
study_widget = widgets.Dropdown(options=studies, value=studies[0],description="Study:")
analyte_widget_1 = widgets.Dropdown(options=analytes_1, value=analytes_1[0],description="Analyte")
analyte_widget_2 = widgets.Dropdown(options=analytes_2, value=analytes_2[0],description="Analyte")
outlier_widget_1 = widgets.RadioButtons(options=['None','Mild','Extreme'], value='None',
                                        description="Outlier", disabled=False)
outlier_widget_2 = widgets.RadioButtons(options=['None','Mild','Extreme'], value='None',
                                        description="Outlier", disabled=False)
def on_study_change(study_accession):
    new_analytes = sorted(elisa_result[elisa_result['study_accession']==study_accession.new]['immunology_symbol'].unique())
    analyte_widget_1.options=new_analytes
study_widget.observe(on_study_change, names='value') 

In [30]:
plot_analyte_by_study = interactive(plot_value_preferred_by_study, { 'manual': False }, 
                      elisa_result = fixed(elisa_result), study_accession = study_widget,
                                    analyte = analyte_widget_1, outlier = outlier_widget_1 )

In [31]:
plot_analyte_by_study

interactive(children=(Dropdown(description='Study:', options=('SDY208', 'SDY21', 'SDY224', 'SDY269', 'SDY28', …

In [32]:
plot_analyte = interactive(plot_value_preferred_by_analyte, { 'manual': False }, 
                      elisa_result = fixed(elisa_result),
                           analyte = analyte_widget_2,
                          outlier = outlier_widget_2)

In [33]:
plot_analyte

interactive(children=(Dropdown(description='Analyte', options=('AFP', 'CCL22', 'CSF3', 'CTACK', 'CXCL9', 'Eota…